# WCS Linking with Glue-Jupyter

This notebook provides an example of linking two dithered NIRISS images using `glue-jupyter`. The development version of `glue-jupyter` is required. To install it:

```
pip install git+https://github.com/glue-viz/glue-jupyter.git
```

The versions used to develop this:

```
# Name                    Version                   Build  Channel
astropy                   3.2rc2                   pypi_0    pypi
bqplot                    0.12.0a1                 pypi_0    pypi
bqplot-image-gl           0.1.4                    pypi_0    pypi
glue-core                 0.15.0.dev5617           pypi_0    pypi
glue-jupyter              0.0.0                    pypi_0    pypi
glue-vispy-viewers        0.12.dev0                pypi_0    pypi
numpy                     1.16.3           py37he5ce36f_0    conda-forge
```

Related issues:

* https://github.com/spacetelescope/jdaviz/issues/25
* https://github.com/glue-viz/glue-jupyter/issues/112

See also:

* http://docs.glueviz.org/en/stable/developer_guide/linking.html

In [ ]:
import warnings

from glue.core.data_factories import load_data
from glue.external.echo import keep_in_sync

from glue_jupyter import jglue

Data files can be downloaded from https://stsci.app.box.com/s/hgj2yuhya3fjwx13s4b2ibuwiqbsmyvu . The example here assumes that you already downloaded the data files into the local working directory, where you are running this notebook from.

In [ ]:
filename_d1 = 'NIRISS_f150w_direct_dit1_nis_cal.fits'
filename_d2 = 'NIRISS_f150w_direct_dit2_nis_cal.fits'

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')  # Silence warnings
    dither1 = load_data(filename_d1)[0]  # 0 = SCI
    dither2 = load_data(filename_d2)[0]

In [ ]:
# Shows the data collection.
dither1

In [ ]:
# Shows the data components for SCI extension.
dither1.components

Now, we create a `glue-jupyter` app and add the data.

In [ ]:
app = jglue()

In [ ]:
app.add_data(dither1)
app.add_data(dither2);

Having loaded both SCI extensions from different dithers and know their WCS components, we can now link them. `add_link()` links the data for overlay.

In [ ]:
app.add_link(dither1, 'Right Ascension', dither2, 'Right Ascension')
app.add_link(dither1, 'Declination', dither2, 'Declination')

We display both dithers in separate, but connected, viewers.

**Note:** For the test image, try setting `percentile` to 99.5%.

In [ ]:
viewer1 = app.imshow(data=dither1)

**Temporary hack:** To sync by WCS, display same image in both viewers; this will be the reference image. Add second image in second viewer. Then, in the second viewer, use the GUI to make the first image layer invisible. (Source: `@astrofrog`)

(Optional) For best effect, set the second viewer to have same `percentile` and `colormap` as the first viewer.

In [ ]:
viewer2 = app.imshow(data=dither1)

In [ ]:
viewer2.add_data(dither2);

Meanwhile, `keep_in_sync` links the viewers' pans.

In [ ]:
# See viewer state parameters
viewer1.state.as_dict()

**Note:** `@astrofrog` advised not to lock down all four corners with auto aspect ratio on or bad things will happen. But looks like aspect is `'equal'` in this case, so probably okay?

**TODO: `keep_in_sync` sometimes stops working, then I have to rerun this cell below, then it works a little and stops again. And sometimes, first viewer would give me the traceback below.**

```
AttributeError: 'NoneType' object has no attribute '_syncing'
```

In [ ]:
keep_in_sync(viewer1.state, 'aspect', viewer2.state, 'aspect')
keep_in_sync(viewer1.state, 'x_min', viewer2.state, 'x_min')
keep_in_sync(viewer1.state, 'x_max', viewer2.state, 'x_max')
keep_in_sync(viewer1.state, 'y_min', viewer2.state, 'y_min')
keep_in_sync(viewer1.state, 'y_max', viewer2.state, 'y_max');